In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
# from pandas_profiling import ProfileReport
from IPython.display import display

In [18]:
print(df.shape)

(17372, 35)


In [20]:
df = pd.read_csv('Final Data/mpii_dataset.csv')
print("Columns: \n\t",df.columns[1:])
print("Number of landmarks:", len(df.columns[2:-3])//2)
print("Except all the landmarks there the Scale column is also important, as it tells the height of a person with respect to 200px")
# print("Lets see how many different catego")

Columns: 
	 Index(['NAME', 'r ankle_X', 'r ankle_Y', 'r knee_X', 'r knee_Y', 'r hip_X',
       'r hip_Y', 'l hip_X', 'l hip_Y', 'l knee_X', 'l knee_Y', 'l ankle_X',
       'l ankle_Y', 'pelvis_X', 'pelvis_Y', 'thorax_X', 'thorax_Y',
       'upper neck_X', 'upper neck_Y', 'head top_X', 'head top_Y', 'r wrist_X',
       'r wrist_Y', 'r elbow_X', 'r elbow_Y', 'r shoulder_X', 'r shoulder_Y',
       'l shoulder_X', 'l shoulder_Y', 'l elbow_X', 'l elbow_Y', 'l wrist_X',
       'l wrist_Y', 'Scale', 'Activity', 'Category'],
      dtype='object')
Number of landmarks: 16
Except all the landmarks there the Scale column is also important, as it tells the height of a person with respect to 200px


In [3]:
print("The number of categories: ",len(np.unique(df['Category'])))
print("The number of activities: ",len(np.unique(df['Activity'])))
print("Unique categories: ",np.unique(df['Category']))

The number of categories:  21
The number of activities:  397
Unique categories:  ['[]' 'bicycling' 'conditioning exercise' 'dancing' 'fishing and hunting'
 'home activities' 'home repair' 'inactivity quiet/light'
 'lawn and garden' 'miscellaneous' 'music playing' 'occupation'
 'religious activities' 'running' 'self care' 'sports' 'transportation'
 'volunteer activities' 'walking' 'water activities' 'winter activities']


In [4]:
df.drop(['Unnamed: 0', 'NAME'], axis = 1 ,inplace = True)
df.head()

,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,l knee_Y,...,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0,620,394,616,269,573,185,647,188,661,221,...,167,692,185,693,240,688,313,3.021046,curling,sports
1,-1,-1,-1,-1,806,543,720,593,-1,-1,...,281,719,299,711,516,545,466,5.641276,curling,sports
2,804,711,816,510,908,438,1040,454,906,528,...,253,1067,253,1167,353,1142,478,4.718488,curling,sports
3,301,461,305,375,201,340,294,342,335,370,...,261,314,264,327,320,362,346,2.681349,curling,sports
4,980,322,896,318,865,248,943,226,948,290,...,147,923,123,995,163,961,223,3.806403,curling,sports


In [5]:
print((df['Category'] == 'dancing').sum())
print((df['Category'] == 'music playing').sum())

697
534


In [6]:
cols = list(df.columns[:-3])
x = []
# y = [] 
d = {}
for i in cols: 
    if i[0]=='r' or i[0]=='l':
        x.append(i[2:-2])
    else: 
        x.append(i[:-2])
# print(x)
for i in x: 
    if i not in d: 
        d[i] = 1
    else: 
        d[i]+=1
for i in d: 
    d[i]//=2
"""
    LEG: ankle->knee->hip * 2
    LOWER BODY: LEG->pelvis *2
     
    ARMS: wrist->elbow->sholder*2
    UPPER BODY: ARMS->thorax *2
    
    HEAD: head top->upper neck
    
    WHOLE BODY: LOWER BODT -> UPPER BODY <- HEAD
    
    Total joints: 4+2+4+2+1+2 = 15
    
    One more thing we can do: 
    Make the feature set as below: 
        L_Leg, R_Leg, L_Arm, R_Arm, Head, LOWER BODY-> UPPER BODY, UPPER BODY-> HEAD
    Thus now we have 8D feature vector, instead of a 32D vector((x,y) of each joint)
"""
print(d)

# thus we have these many landmark, but we will later remove some of them

{'ankle': 2, 'knee': 2, 'hip': 2, 'pelvis': 1, 'thorax': 1, 'upper neck': 1, 'head top': 1, 'wrist': 2, 'elbow': 2, 'shoulder': 2}


In [7]:
df_bicycling = df.loc[df["Category"] == 'bicycling']
df_walking = df.loc[df["Category"] == 'walking']
print(df_bicycling.shape, df_walking.shape)

(516, 35) (577, 35)


# Making the traing data

In [8]:
df_bicycling.head()

,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,l knee_Y,...,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
113,-1,-1,-1,-1,823,845,1056,851,-1,-1,...,438,1105,454,1111,671,1025,876,6.291709,"bicycling, general",bicycling
114,-1,-1,1480,1049,1513,815,1314,698,1242,863,...,439,1329,391,1242,566,1110,758,6.025147,"bicycling, general",bicycling
115,940,606,879,418,752,322,679,292,818,482,...,112,788,112,864,168,960,206,3.319036,"bicycling, general",bicycling
250,536,844,536,682,542,473,427,456,414,620,...,403,447,391,391,529,405,607,2.717199,"bicycling, racing and road",bicycling
251,1158,534,1148,431,1223,207,1132,212,1043,456,...,143,1009,171,970,289,928,389,3.529877,"bicycling, mountain",bicycling


In [9]:
df_bicycling.columns

Index(['r ankle_X', 'r ankle_Y', 'r knee_X', 'r knee_Y', 'r hip_X', 'r hip_Y',
       'l hip_X', 'l hip_Y', 'l knee_X', 'l knee_Y', 'l ankle_X', 'l ankle_Y',
       'pelvis_X', 'pelvis_Y', 'thorax_X', 'thorax_Y', 'upper neck_X',
       'upper neck_Y', 'head top_X', 'head top_Y', 'r wrist_X', 'r wrist_Y',
       'r elbow_X', 'r elbow_Y', 'r shoulder_X', 'r shoulder_Y',
       'l shoulder_X', 'l shoulder_Y', 'l elbow_X', 'l elbow_Y', 'l wrist_X',
       'l wrist_Y', 'Scale', 'Activity', 'Category'],
      dtype='object')

# Lets just explore some points related to landmarks: 
- Each landmark has x and y co-ordinate
- If a part of body is not visible in the photo then the co-ordinates are marked as `-1`

In [10]:
# lets also see how many joints are missing: 
(df_walking.iloc[:,:32] < 0).sum()

r ankle_X       190
r ankle_Y       190
r knee_X         92
r knee_Y         92
r hip_X          13
r hip_Y          13
l hip_X          12
l hip_Y          12
l knee_X         87
l knee_Y         87
l ankle_X       182
l ankle_Y       182
pelvis_X         15
pelvis_Y         15
thorax_X          1
thorax_Y          1
upper neck_X      1
upper neck_Y      1
head top_X        1
head top_Y       14
r wrist_X         8
r wrist_Y         8
r elbow_X         4
r elbow_Y         4
r shoulder_X      1
r shoulder_Y      1
l shoulder_X      1
l shoulder_Y      1
l elbow_X         5
l elbow_Y         5
l wrist_X        10
l wrist_Y        10
dtype: int64

In [11]:
(df_bicycling.iloc[:,:32] < 0).sum()

r ankle_X       58
r ankle_Y       58
r knee_X        18
r knee_Y        18
r hip_X          3
r hip_Y          3
l hip_X          2
l hip_Y          2
l knee_X        19
l knee_Y        19
l ankle_X       56
l ankle_Y       56
pelvis_X         4
pelvis_Y         4
thorax_X         1
thorax_Y         1
upper neck_X     1
upper neck_Y     1
head top_X       1
head top_Y       6
r wrist_X        6
r wrist_Y        6
r elbow_X        2
r elbow_Y        2
r shoulder_X     0
r shoulder_Y     0
l shoulder_X     0
l shoulder_Y     0
l elbow_X        1
l elbow_Y        1
l wrist_X        4
l wrist_Y        4
dtype: int64

In [12]:
## new df: 
# each joint will have 2 lists one for left and other for right

def containZero(x):
    p = np.asarray(x)
    v = (p>0).sum()
    if v==len(x):
        return False
    return True
def dist(p1, p2):
    s = 0 
    if containZero(p1) or containZero(p2):
        return 0
    for i in range(len(p1)): 
        s+=(p1[i]-p2[i])**2
    return s**0.5 
def angle_length(p1, p2):
    '''
    Input:
    p1 - coordinates of point 1. List
    p2 - coordinates of point 2. List
    Output:
    Tuple containing the angle value between the line formed by two input points 
    and the x-axis as the first element and the length of this line as the second
    element
    '''

    angle = math.atan2(- int(p2[0]) + int(p1[0]), int(p2[1]) - int(p1[1])) * 180.0 / np.pi
    length = math.hypot(int(p2[1]) - int(p1[1]), - int(p2[0]) + int(p1[0]))

    return round(angle), round(length)

def csvMaker(csv):
    noSamples = csv.shape[0]
    new_cols = csv.columns
    scale = csv['Scale']
    column_mapping = {column:i for i,column in enumerate(csv.columns)}
    ankle_knee = [[],[]]
    knee_hip = [[],[]]
#     hip_pelvis = [[],[]]

    wrist_elbow = [[],[]]
    elbow_shoulder = [[],[]]
#     shoulder_thorax = [[],[]]

#     pelvis_thorax = []
    
    for i in range(csv.shape[0]):
        l_ankle_x = csv.iloc[i,column_mapping['l ankle_X']]
        l_ankle_y = csv.iloc[i,column_mapping['l ankle_Y']]

        l_knee_x = csv.iloc[i, column_mapping['l knee_X']]
        l_knee_y = csv.iloc[i, column_mapping['l knee_Y']]

        l_hip_x = csv.iloc[i, column_mapping['l hip_X']]
        l_hip_y = csv.iloc[i, column_mapping['l hip_Y']]

        #########################
#         pelvis_x = csv.iloc[i, column_mapping['pelvis_X']]
#         pelvis_y = csv.iloc[i, column_mapping['pelvis_Y']]
        #########################

        r_ankle_x = csv.iloc[i,column_mapping['r ankle_X']]
        r_ankle_y = csv.iloc[i,column_mapping['r ankle_Y']]

        r_knee_x = csv.iloc[i, column_mapping['r knee_X']]
        r_knee_y = csv.iloc[i, column_mapping['r knee_Y']]

        r_hip_x = csv.iloc[i, column_mapping['r hip_X']]
        r_hip_y = csv.iloc[i, column_mapping['r hip_Y']]

        #Lower body parts ends here

        l_wrist_x = csv.iloc[i, column_mapping['l wrist_X']]
        l_wrist_y = csv.iloc[i, column_mapping['l wrist_Y']]

        l_elbow_x = csv.iloc[i, column_mapping['l elbow_X']]
        l_elbow_y = csv.iloc[i, column_mapping['l elbow_Y']]

        l_shoulder_x = csv.iloc[i, column_mapping['l shoulder_X']]
        l_shoulder_y = csv.iloc[i, column_mapping['l shoulder_Y']]

        #########################
#         thorax_y = csv.iloc[i, column_mapping['thorax_X']]
#         thorax_x = csv.iloc[i, column_mapping['thorax_Y']]
        #########################

        r_wrist_x = csv.iloc[i, column_mapping['r wrist_X']]
        r_wrist_y = csv.iloc[i, column_mapping['r wrist_Y']]

        r_elbow_x = csv.iloc[i, column_mapping['r elbow_X']]
        r_elbow_y = csv.iloc[i, column_mapping['r elbow_Y']]

        r_shoulder_x = csv.iloc[i, column_mapping['r shoulder_X']]
        r_shoulder_y = csv.iloc[i, column_mapping['r shoulder_Y']]

        """
            Now distances will be cmputed
                ankle_knee = [[],[]]
                knee_hip = [[],[]]
                hip_pelvis = [[],[]]

                wrist_elbow = [[],[]]
                elbow_shoulder = [[],[]]
                shoulder_thorax = [[],[]]

                pelvis_thorax = []
        """
        ankle_knee[0].append(dist([l_ankle_x, l_ankle_y], [l_knee_x, l_knee_y]))
        ankle_knee[1].append(dist([r_ankle_x, r_ankle_y], [r_knee_x, r_knee_y]))

        knee_hip[0].append(dist([l_knee_x, l_knee_y], [l_hip_x, l_hip_y]))
        knee_hip[1].append(dist([r_knee_x, r_knee_y], [r_hip_x, r_hip_y]))

#         hip_pelvis[0].append(dist([l_hip_x, l_hip_y], [pelvis_x, pelvis_y]))
#         hip_pelvis[1].append(dist([r_hip_x, r_hip_y], [pelvis_x, pelvis_y]))

        wrist_elbow[0].append(dist([l_wrist_x, l_wrist_y], [l_elbow_x, l_elbow_y]))
        wrist_elbow[1].append(dist([r_wrist_x, r_wrist_y], [r_elbow_x, r_elbow_y]))

        elbow_shoulder[0].append(dist([l_elbow_x, l_elbow_y], [l_shoulder_x, l_shoulder_y]))
        elbow_shoulder[1].append(dist([r_elbow_x, r_elbow_y], [r_shoulder_x, r_shoulder_y]))
        
#         shoulder_thorax[0].append(dist([l_shoulder_x, l_shoulder_y], [thorax_x, thorax_y]))
#         shoulder_thorax[1].append(dist([r_shoulder_x, r_shoulder_y], [thorax_x, thorax_y]))
        
#         pelvis_thorax.append(dist([pelvis_x, pelvis_y], [thorax_x, thorax_y]))
        
    ankle_knee = np.asarray(ankle_knee)
    knee_hip = np.asarray(knee_hip)
#     hip_pelvis = np.asarray(hip_pelvis)
    wrist_elbow = np.asarray(wrist_elbow)
    elbow_shoulder = np.asarray(elbow_shoulder)
#     shoulder_thorax = np.asanyarray(shoulder_thorax)
#     pelvis_thorax = np.asarray(pelvis_thorax)


    dict_13 = {'l ankle_knee': ankle_knee[0], 'r ankle_knee': ankle_knee[1], 'l knee_hip': knee_hip[0], 'r knee_hip': knee_hip[1],
             'l wrist_elbow':wrist_elbow[0], 'r wrist_elbow': wrist_elbow[1], 'l elbow_shoulder': elbow_shoulder[0], 'r elbow_shoulder': elbow_shoulder[1],
             'scale': scale
            }
    
    dict_high_Features = {'l leg': ankle_knee[0]+knee_hip[0], 'r leg': ankle_knee[1]+knee_hip[1],
                          'l hand': wrist_elbow[0]+elbow_shoulder[0], 'r hand': wrist_elbow[1]+elbow_shoulder[1],
                          'scale': scale
                         }
    return pd.DataFrame(dict_13), pd.DataFrame(dict_high_Features)

In [13]:
newBicycling = csvMaker(df_bicycling)
display(newBicycling[0].head())
display(newBicycling[1].head())

,l ankle_knee,r ankle_knee,l knee_hip,r knee_hip,l wrist_elbow,r wrist_elbow,l elbow_shoulder,r elbow_shoulder,scale
113,0.000000,0.000000,0.000000,0.000000,222.308344,200.000000,217.082933,253.550784,6.291709
114,135.830777,0.000000,180.024998,236.315467,232.997854,153.117602,195.432853,253.639508,6.025147
115,92.227978,197.648678,235.416652,159.201131,103.247276,137.974635,94.403390,145.437272,3.319036
250,96.337947,162.000000,164.514437,209.086107,79.246451,67.911707,148.929514,117.686023,2.717199
251,228.569464,103.484298,259.724854,236.222353,108.461975,99.724621,124.277914,88.238314,3.529877


,l leg,r leg,l hand,r hand,scale
113,0.000000,0.000000,439.391278,453.550784,6.291709
114,315.855775,236.315467,428.430707,406.757110,6.025147
115,327.644630,356.849809,197.650666,283.411907,3.319036
250,260.852384,371.086107,228.175965,185.597730,2.717199
251,488.294319,339.706651,232.739889,187.962935,3.529877


In [14]:
newWalking = csvMaker(df_walking)
# newWalking[0]['1'] = [1]*newWalking[0].shape[0]
display(newWalking[0].head())
display(newWalking[1].head())

,l ankle_knee,r ankle_knee,l knee_hip,r knee_hip,l wrist_elbow,r wrist_elbow,l elbow_shoulder,r elbow_shoulder,scale
245,0.000000,0.000000,0.000000,0.000000,180.568547,209.308385,236.171548,214.560015,6.418060
275,131.061054,0.000000,169.106475,188.042548,95.021050,105.000000,120.702113,129.027129,3.158167
276,131.973482,131.548470,158.050625,173.415686,55.317267,123.693169,137.186005,102.239914,2.860937
447,125.447200,130.552671,116.760439,108.060168,83.408633,91.197588,100.806746,83.600239,2.852872
529,82.219219,82.969874,105.076163,106.117859,51.478151,71.063352,71.470274,79.227520,1.837842


,l leg,r leg,l hand,r hand,scale
245,0.000000,0.000000,416.740094,423.868400,6.418060
275,300.167530,188.042548,215.723163,234.027129,3.158167
276,290.024107,304.964156,192.503272,225.933083,2.860937
447,242.207639,238.612840,184.215378,174.797827,2.852872
529,187.295382,189.087733,122.948424,150.290872,1.837842


In [15]:
newBicycling[0].to_csv('Final Data/Bicycle-8.csv', index = False)
newBicycling[1].to_csv('Final Data/Bicycle-4.csv', index = False)

In [16]:
newWalking[0].to_csv('Final Data/Walking-8.csv', index = False)
newWalking[1].to_csv('Final Data/Walking-4.csv', index = False)

In [17]:
df.iloc[:,-3].mean()

3.6052496470083115

In [22]:
newWalking[0].columns[:-1]

Index(['l ankle_knee', 'r ankle_knee', 'l knee_hip', 'r knee_hip',
       'l wrist_elbow', 'r wrist_elbow', 'l elbow_shoulder',
       'r elbow_shoulder'],
      dtype='object')